<a href="https://colab.research.google.com/github/Sai0472003/PatentLitigation_Impact/blob/main/DeepIV_Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [49]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from google.colab import drive
drive.mount('/content/drive')

data = pd.read_csv('/content/drive/My Drive/ECO225_Final_Project/DATA/rnd_iv.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [50]:
clean_data = data[['closed_plus_1_rd_sale', 'disp_dismissed', 'defendant',
             'dismissed_settlement', 'dismissed_voluntary', 'disp_before_issue_joined',
             'filed_minus_1_rd_sale', 'case_length', 'info_tech', 'health_care',
             'materials', 'industrials', 'consumer_discretionary', 'state_province',
             'DISTRICT', 'date_filed', 'probability']]

clean_data['year_filed'] = pd.to_datetime(clean_data['date_filed']).dt.year
clean_data = clean_data.drop('date_filed', axis=1)
clean_data = pd.get_dummies(clean_data, columns=['state_province', 'DISTRICT', 'year_filed'])
clean_data[clean_data.select_dtypes(include='bool').columns] = clean_data.select_dtypes(include='bool').astype(int)
clean_data.head()

<ipython-input-50-f733b0ce2af6>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_data['year_filed'] = pd.to_datetime(clean_data['date_filed']).dt.year


,closed_plus_1_rd_sale,disp_dismissed,defendant,dismissed_settlement,dismissed_voluntary,disp_before_issue_joined,filed_minus_1_rd_sale,case_length,info_tech,health_care,...,year_filed_2011,year_filed_2012,year_filed_2013,year_filed_2014,year_filed_2015,year_filed_2016,year_filed_2017,year_filed_2018,year_filed_2019,year_filed_2020
0,0.100666,1,1.0,0,0,0,0.118179,14,0,1,...,1,0,0,0,0,0,0,0,0,0
1,0.125455,1,1.0,1,0,0,0.110958,9,0,0,...,0,0,0,0,1,0,0,0,0,0
2,0.000000,1,1.0,1,0,0,0.000000,6,0,0,...,1,0,0,0,0,0,0,0,0,0
3,0.050520,1,1.0,0,1,0,0.048885,2,0,1,...,0,0,0,0,0,0,1,0,0,0
4,0.137757,0,1.0,0,0,1,0.122804,4,0,0,...,0,0,1,0,0,0,0,0,0,0


In [73]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from google.colab import drive
drive.mount('/content/drive')

data = pd.read_csv('/content/drive/My Drive/ECO225_Final_Project/DATA/rnd_iv.csv')

clean_data = data[['closed_plus_1_rd_sale', 'disp_dismissed', 'defendant',
             'dismissed_settlement', 'dismissed_voluntary', 'disp_before_issue_joined',
             'filed_minus_1_rd_sale', 'case_length', 'info_tech', 'health_care',
             'materials', 'industrials', 'consumer_discretionary', 'state_province',
             'DISTRICT', 'date_filed', 'probability']]

clean_data['year_filed'] = pd.to_datetime(clean_data['date_filed']).dt.year
clean_data = clean_data.drop('date_filed', axis=1)
clean_data = pd.get_dummies(clean_data, columns=['state_province', 'DISTRICT', 'year_filed'])
clean_data[clean_data.select_dtypes(include='bool').columns] = clean_data.select_dtypes(include='bool').astype(int)
clean_data.head()

from sklearn.preprocessing import StandardScaler
from econml.iv.nnet import DeepIV
import keras
import tensorflow as tf # Import tensorflow
from tensorflow.keras.layers import Input, Concatenate # Import necessary layers

# Define outcome, treatment, instrument, covariates
Y = clean_data["closed_plus_1_rd_sale"].values                           # outcome
T = clean_data["disp_dismissed"].values.reshape(-1, 1)                    # treatment
Z = clean_data["probability"].values.reshape(-1, 1)                       # instrument

X_cols = clean_data.columns.difference(["closed_plus_1_rd_sale", "disp_dismissed",
                                        'probability'])
X = clean_data[X_cols].values                                            # covariates for heterogeneity

# Scale X for neural nets
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


import tensorflow as tf
from tensorflow.keras import layers, models
from econml.iv.nnet import DeepIV

# Define the treatment model
def build_treatment_model(input_dim):
    model = models.Sequential([
        layers.Input(shape=(input_dim,)),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.17),
        layers.Dense(64, activation='relu'),
        layers.Dropout(0.17),
        layers.Dense(32, activation='relu'),
        layers.Dropout(0.17),
        layers.Dense(1)  # Output layer for treatment prediction
    ])
    return model

# Define the response model
def build_response_model(input_dim):
    model = models.Sequential([
        layers.Input(shape=(input_dim,)),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.17),
        layers.Dense(64, activation='relu'),
        layers.Dropout(0.17),
        layers.Dense(32, activation='relu'),
        layers.Dropout(0.17),
        layers.Dense(1)  # Output layer for response prediction
    ])
    return model

# Assume Z, X, T, and Y are your data arrays
# Ensure they are NumPy arrays
Z = Z.to_numpy() if hasattr(Z, 'to_numpy') else Z
X = X.to_numpy() if hasattr(X, 'to_numpy') else X
T = T.to_numpy() if hasattr(T, 'to_numpy') else T
Y = Y.to_numpy() if hasattr(Y, 'to_numpy') else Y

# Determine input dimensions
input_dim_m = Z.shape[1] + X.shape[1]
input_dim_h = T.shape[1] + X.shape[1]

# Instantiate the DeepIV estimator
est = DeepIV(
    n_components=10,  # Number of Gaussians in the mixture density networks
    m=lambda z, x: build_treatment_model(input_dim_m),
    h=lambda t, x: build_response_model(input_dim_h),
    n_samples=1  # Number of samples used to estimate the response
)

# Fit the model
est.fit(Y, T, X=X, Z=Z)

# Estimate treatment effects
treatment_effects = est.effect(X_test)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<ipython-input-73-7f56a6cdd133>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_data['year_filed'] = pd.to_datetime(clean_data['date_filed']).dt.year


AttributeError: 'Sequential' object has no attribute 'shape'

In [65]:
# Fit the model
deepiv.fit(Y, T, X=X_scaled, Z=Z, inference=None)
treatment_effects = deepiv.effect(X_test)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


AttributeError: module 'keras.api.backend' has no attribute 'int_shape'

In [ ]:
import matplotlib.pyplot as plt

# Plot the distribution of estimated CATEs as a histogram (bar‐style)
plt.figure(figsize=(8, 5))
plt.hist(tau_hat, bins=40, edgecolor='black')
plt.title("Distribution of Estimated CATEs")
plt.xlabel("Estimated CATE")
plt.ylabel("Frequency")
plt.tight_layout()
plt.show()
